# Polycube robustness

In [1]:
import robustness
import altair as alt
import pandas as pd
import utils
import pickle

Let's start by calculating $N_g$, the set of genotype neighbours one point mutation away from a given genotype $g$. Depending on the allowed amount of colors,input size and dimentionality, the corresponding neigbourhood will be of different sizes, as we shall see later. 

As an initial example, this is the number of 2D mutations for the genotype of a 2x2 square, if we allow only one color and one cube type:

In [2]:
', '.join(robustness.enumerateMutations('040087000000', maxColor=1, maxCubes=1, dim=2))

'840087000000, 000087000000, 048487000000, 040487000000, 040003000000, 040007000000, 040087840000, 040087040000, 040087008400, 040087000400, 040087000084, 040087000004'

## Genotype robustness

We define the genotype robustness $\rho_g$ of a genotype $g$ as:
$$ \rho_g = \frac{1}{\left | N_g \right |}\sum_{n \in N_g} \begin{cases}
 1 & \text{ if } p(n)=p(g) \\ 
 0 & \text{ otherwise } 
\end{cases} $$
where $N_g$ is the set of 1-mutant neigbours of $g$ and $p(g)$ is the phenotype assembled from genotype $g$.

Let's calculate the robustness for the same genotype as before. This time in the default 3D with a maximum of 3 colours and 2 cube types

In [3]:
robustness.calcGenotypeRobustness('040087000000', 3, 2)

0.7614678899082569

 If we allow a larger phenotype space the robustness will be higher, since there is more room for neutral mutations:

In [4]:
genotypeRobustnessData = []
for maxCol in range(1,10):
    for maxCubes in range(1,10):
        genotypeRobustnessData.append({
            'maxCol': maxCol,
            'maxCubes': maxCubes,
            'robustness': robustness.calcGenotypeRobustness('040087000000', maxCol, maxCubes)
        })
alt.Chart(pd.DataFrame(data=genotypeRobustnessData)).mark_rect().encode(
    alt.X('maxCol:O', title="Allowed colors"),
    alt.Y('maxCubes:O', title="Allowed cube types"),
    alt.Color('robustness', scale=alt.Scale(domain=(0,1))),
)

alt.Chart(...)

By increasing the mutation radius from 1 to n, we can calculate the n-robustness $\rho_g^{(n)}$ of a genotype $g$,  meaning the fraction of mutational neigbours within n point mutations that map to the same phenotype as g. With increasing n, $\rho_g^{(n)}$ approaches the frequency of g in the whole genome space.

In [19]:
robustnessRadiiData = []
for r in range(1,5):
    nrob = robustness.calcGenotypeRobustness('040087000000', 2, 2, 2, radius=r)
    print(nrob)
    robustnessRadiiData.append({
        'radius': r,
        'robustness': nrob
    })

0.673469387755102
0.35985533453887886
0.15441854339523245
0.06913026933322058


In [21]:
alt.Chart(pd.DataFrame(data=robustnessRadiiData)).mark_line().encode(
    alt.X('radius', title="Radius", axis=alt.Axis(tickMinStep = 1)),
    alt.Y('robustness', title="Robustness")
)

alt.Chart(...)

## Phenotype robustness

With the genotype robustness defined, we can then define the phenotype robustness $\rho_p$ as the average genotype robustness for all genotypes enconding for the given phenotype:
$$ \rho_p = \frac{1}{\left | P \right |} \sum_{g \in P}\rho_g $$
where $P$ is the set of all genotypes with phenotype $p$.

Plot the distribution of the genotype robustness values for a few key phenotypes. How large is the variance? Is the distribution gaussian or is it skewed any way? (If the variance is low, no need to sample as many genotypes to get a good estimation of the mean)

In [22]:
phenos = utils.loadPhenos('../cpp/out/3d_31c_5t_1e7/phenos')

In [45]:
[(i, pheno['count']) for i, pheno in enumerate(phenos) if pheno['count'] > 3]

[(8, 82689),
 (9, 38670),
 (10, 27099),
 (11, 19757),
 (12, 19634),
 (13, 19507),
 (14, 18398),
 (15, 5080),
 (16, 22),
 (17, 6),
 (21, 191),
 (22, 178),
 (23, 168),
 (24, 157),
 (25, 68),
 (26, 49),
 (27, 45),
 (28, 42),
 (29, 40),
 (30, 38),
 (31, 33),
 (32, 31),
 (33, 25),
 (34, 24),
 (35, 23),
 (36, 23),
 (37, 22),
 (38, 19),
 (39, 19),
 (40, 19),
 (41, 19),
 (42, 19),
 (43, 18),
 (44, 17),
 (45, 17),
 (46, 16),
 (47, 16),
 (48, 15),
 (49, 15),
 (50, 15),
 (51, 15),
 (52, 15),
 (53, 15),
 (54, 14),
 (55, 14),
 (56, 14),
 (57, 13),
 (58, 13),
 (59, 13),
 (60, 13),
 (61, 13),
 (62, 13),
 (63, 13),
 (64, 13),
 (65, 13),
 (66, 13),
 (67, 12),
 (68, 12),
 (69, 12),
 (70, 12),
 (71, 12),
 (72, 12),
 (73, 12),
 (74, 11),
 (75, 11),
 (76, 11),
 (77, 11),
 (78, 11),
 (79, 11),
 (80, 11),
 (81, 11),
 (82, 11),
 (83, 11),
 (84, 11),
 (85, 10),
 (86, 10),
 (87, 10),
 (88, 10),
 (89, 10),
 (90, 10),
 (91, 10),
 (92, 10),
 (93, 9),
 (94, 9),
 (95, 9),
 (96, 9),
 (97, 9),
 (98, 9),
 (99, 9),
 (10

In [ ]:
# Generating Data
data = []
for phenoName, phenoIdx in [('a',21), ('b',22), ('c',23)]:
    p = phenos[phenoIdx]
    for genotype in p['genotypes']:
        data.append({'r': robustness.calcGenotypeRobustness(genotype), 'pheno': phenoName})
source = pd.DataFrame(data=data)

In [ ]:
alt.Chart(source).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'
).mark_bar(opacity=0.5).encode(
    alt.X('r:Q', bin=True),
    alt.Y('sum(pct):Q', axis=alt.Axis(format='%')),
    color="pheno:O"
) + base.mark_rule(color='red').encode(
    x='mean(r):Q',
    size=alt.value(5),
    color="pheno:O"
)

In [ ]:
for pheno in ['a', 'b', 'c']:
    df = source.loc[source['pheno'] == pheno]
    print("{}: mean: {:.2}, variance: {:.2}".format(pheno, df.mean()[0], df.var()[0]))

For a large dataset, this will take a while; so it's better to calculate it separately by running `python robustness.py` and saving the pickled data:
In this case, we load a random sample of 100 phenotypes:

In [13]:
#phenotypeRobustnessData = robustness.calcPhenotypeRobustness(path='../cpp/out/3d', sampleSize=100)
phenotypeRobustnessData = pickle.load(open('../cpp/out/3d_31c_5t_1e7/robustness.p', "rb"))

In [14]:
phenotypeRobustnessData

,freq,robustness,rule
0,1.160000e-05,0.458722,000000918f04000e00869200880000000000001109000000
1,4.300000e-06,0.455385,0000058a0e0f8c00000000000000008c000000000e850900
2,2.200000e-06,0.473333,000000070c110000000000858d0693000089000900000000
3,1.600000e-06,0.458686,8800860000000000058c9000000000008612000a000000...
4,1.600000e-06,0.444583,000400000d0b0000000000908a00008d0000118600100000
5,1.500000e-06,0.469265,0b00110e0b0088000000000000000086000093040000008f
6,1.500000e-06,0.458085,000084118d0000000000008a000089000000910007008d...
7,1.400000e-06,0.474982,0000890000000c00000000000a00078f130091860000008c
8,1.300000e-06,0.466114,00870b008f00890e86120000009000000000000007000000
9,1.200000e-06,0.452778,0000008e1087000f000000009100058b00000000008900...


In [15]:
alt.Chart(phenotypeRobustnessData).transform_calculate(
        url='https://akodiat.github.io/polycubes?hexRule=' + alt.datum.rule
    ).mark_circle(size=60).encode(
        alt.X('freq', scale=alt.Scale(type='log'), title="Phenotype frequency"),
        alt.Y('robustness', title="Phenotype robustness"),
        href='url:N',
        tooltip=['rule']
    ).interactive()

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.Chart(...)